In [2]:
import pandas as pd
import datetime
import time
import numpy as np
from operator import attrgetter

In [20]:
# Создаем все df
df_customers = pd.read_csv('olist_customers_dataset.csv', encoding='UTF-8', sep=',')
df_order = pd.read_csv('olist_orders_dataset.csv', encoding='UTF-8',sep=',')
df_items = pd.read_csv('olist_order_items_dataset.csv',encoding='UTF-8', sep=',')

0.Для решения задачи проведи предварительное исследование данных и сформулируй, что должно считаться покупкой. Обосновать свой выбор ты можешь с помощью фактов оплат, статусов заказов и других имеющихся данных

In [4]:
# Посмотрим на обьем
df_order.shape

(99441, 8)

In [5]:
# Посмотрим на типы
df_order.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [6]:
# Помотрим пустые значения
df_order.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [7]:
# Просто посмотрим
df_order.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [8]:
# Посмотрим разбивку по статусам
df_order.value_counts('order_status')

order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
dtype: int64

In [9]:
# Дальше работаем со статусами, где нам необходимо выбрать нужный/нужные для покупок
# df_order.order_approved_at - главная столбец для нас(время оплаты подтверждения оплаты заказа)
# Мы не возьмем df_order.order_approved_at с пустыми значениями, посмотрим
df_order.query('order_approved_at.isnull()==True').order_status.unique()
# Делаем вывод, что canceled, created - не берем, но сюда попал еще delivered
# Посмотрим что там
df_order.query('order_approved_at.isnull()==True').query('order_status=="delivered"').count()
# 14 значений, рассмотрим это как аномалию, будем ее учитывать. но delivered точно берем. Заказ доставлен значит был опачен
# unavailable —  недоступен Не берем, не будем рассматривать данный статус как покупку
# approved - возьмем, конечно он странный но дата подтвердения оплаты есть
# processing —  в процессе сборки заказа, берем статус оплаты есть сборка пошла(заказ скорее всего оплачен)
# shipped —  отгружен со склада, берем статус оплаты есть груз отгружен(заказ скорее всего оплачен)
# delivered —  доставлен пользователю, берем все(бесплатно заказы не раздаем)
df_order.query('order_status=="invoiced"')
# invoiced - очень спорная позиция, статус оплачен, но называется как выставлен счет(исходя из описания брать не будем)
# Итого взяли ["delivered","shipped","processing"]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaN,NaN,2017-05-09 00:00:00
103,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,2018-08-03 17:44:42,2018-08-07 06:15:14,NaN,NaN,2018-08-21 00:00:00
455,38b7efdf33dd5561f4f5d4f6e07b0414,021e84751ba0ead75b6d314a6ead88d9,invoiced,2017-08-01 18:17:41,2017-08-01 18:32:30,NaN,NaN,2017-08-28 00:00:00
1833,51b0dccc8596ce37a930dff2d63a10a2,31bf1057b00f14804278590bbac18b1b,invoiced,2017-05-05 22:34:48,2017-05-05 22:45:12,NaN,NaN,2017-06-06 00:00:00
2116,5504eaa5a86eb25fa666cf2e6b96c701,a30be38e1ed0ffb39b318bf1d15e98d9,invoiced,2017-11-29 08:33:06,2017-11-29 08:56:23,NaN,NaN,2017-12-20 00:00:00
...,...,...,...,...,...,...,...,...
98414,41ad2ae32ee66c07ef3b0301ba02ad27,5c7f8d14a0a12198d1d07167d7d343b3,invoiced,2017-11-24 15:54:47,2017-11-24 19:33:24,NaN,NaN,2017-12-18 00:00:00
98624,9ffb325177b96c422447e3b9ee9a6270,e4079117ea0dc8bc27274e34c4796515,invoiced,2018-05-10 19:11:18,2018-05-10 19:37:27,NaN,NaN,2018-06-05 00:00:00
98845,021dac6e3120485db6a77474aa21b979,c40f7f00e91eb8a0da1e58484380a796,invoiced,2017-11-24 22:46:09,2017-11-25 01:53:42,NaN,NaN,2017-12-15 00:00:00
99031,c526fe83c01c48066394ff5d81f9ab9a,6c678c38f3a0874b7bfeeb2d5fb22876,invoiced,2018-08-07 11:11:30,2018-08-07 11:25:01,NaN,NaN,2018-08-17 00:00:00


1.Сколько у нас пользователей, которые совершили покупку только один раз?

In [10]:
# Смерджим таблицы, будем отталкиваться от пользовательской таблицы
df_order_customers = df_order.merge(df_customers, how='inner',on='customer_id')
# Будем считать за покупку все, что не отменено и оплачено
one_buy_customers = df_order_customers.query('order_status in ["delivered","shipped","processing"]')
# Посчитаем уникальных пользователей сделавших покупки
one_buy_customers_unique = one_buy_customers.groupby('customer_unique_id',as_index=False)\
                          .agg({'customer_city':'count'})\
                          .shape
# Проверим все ли сделали по одной покупке
one_buy_customers_unique_1 = one_buy_customers.groupby('customer_unique_id',as_index=False)\
                            .agg({'customer_city':'count'})\
                            .query('customer_city == 1')\
                            .shape
# Ответ выбираем наименьший из 2-х последних шагов
one_buy_customers_unique_1[0]

91814

2.Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)?

In [20]:
# Забираем в df только полностью не доставленные заказы
ne_dostavleno = df_order.query('order_status == "canceled" or order_status == "unavailable"')
# За месяц "недоставки" возьмем обещанную дату order_estimated_delivery_date 
ser_null = ne_dostavleno.order_estimated_delivery_date.isnull().sum() # Делаем вывод, что она везде есть ser_null=0
# Меняем на дату
ne_dostavleno.order_estimated_delivery_date = ne_dostavleno.order_estimated_delivery_date.astype('datetime64[ns]')
# Cоздаем период
ne_dostavleno.order_estimated_delivery_date = ne_dostavleno.order_estimated_delivery_date.dt.strftime('%Y-%m')
# Не доставлено по каждой из причин(ответ)
ne_dostavleno_cat_mean = ne_dostavleno.groupby(['order_estimated_delivery_date','order_status'], as_index= False)\
.agg({'order_id':'count'})\
.groupby('order_status', as_index=False).agg({'order_id':'mean'})
# Ответ
ne_dostavleno_cat_mean

,order_status,order_id
0,canceled,24.038462
1,unavailable,29.000000


3.По каждому товару определить, в какой день недели товар чаще всего покупается.

In [38]:
# Начнем с мерджа
df_order_items = df_order.merge(df_items, how='inner', on='order_id')
# Отберем заказы которые были куплены, оставляем дату подтверждеия оплаты
df_order_items = df_order_items\
            .query('order_status in ["delivered","shipped","processing"]')\
            .groupby(['product_id','order_purchase_timestamp'], as_index=False).agg({'customer_id':'count'})
# Делаем из даты день недели
df_order_items.order_purchase_timestamp = df_order_items.order_purchase_timestamp.astype('datetime64[ns]')
df_order_items.order_purchase_timestamp = df_order_items.order_purchase_timestamp.dt.day_name()
# Групируем еще раз
df_pivot = df_order_items.groupby(['order_purchase_timestamp','product_id'],as_index=False).agg({'customer_id':'sum'})\
.pivot(index='product_id',columns='order_purchase_timestamp',values='customer_id').fillna(0)
# Получаем самый покупаемый день для каждого товара максимальный # df_pivot.max(axis=1)
df_pivot.idxmax(axis=1)

product_id
00066f42aeeb9f3007548bb9d3f33c38       Sunday
00088930e925c41fd95ebfe695fd2655      Tuesday
0009406fd7479715e4bef61dd91f2462     Thursday
000b8f95fcb9e0096488278317764d19       Friday
000d9be29b5207b54e86aa1b1ac54872      Tuesday
                                      ...    
fff6177642830a9a94a0f2cba5e476d1     Saturday
fff81cc3158d2725c0655ab9ba0f712c       Monday
fff9553ac224cec9d15d49f5a263411f       Friday
fffdb2d0ec8d6a61f0a0a0db3f25b441      Tuesday
fffe9eeff12fcbd74a2f2b007dde0c58    Wednesday
Length: 32614, dtype: object

4.Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? 
Не стоит забывать, что внутри месяца может быть не целое количество недель. 
Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть.

In [26]:
df_order_users = df_order.merge(df_customers, how='inner', on='customer_id')
# Отбираем покупки
df_order_users = df_order_users.query('order_status in ["delivered","shipped","processing"]')
# Делаем месяц
df_order_users['order_purchase_timestamp'] = pd.to_datetime(df_order_users['order_purchase_timestamp']).dt.strftime('%Y-%m')
# Выводим количество покупок на каждого пользователся в месяц и делим на среднее количество недель в месяце
df_user_bought = df_order_users\
    .groupby(['customer_unique_id', 'order_purchase_timestamp'],as_index=False)\
    .agg({'customer_id':'count'})\
    .groupby(['customer_unique_id', 'order_purchase_timestamp'],as_index=False)\
    .agg({'customer_id': lambda x: x/4.35})
# Находим среднее и выводим ответ
df_user_bought['customer_id'] = df_user_bought['customer_id'].mean()
df_user_bought.head()

,customer_unique_id,order_purchase_timestamp,customer_id
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,0.233044
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05,0.233044
2,0000f46a3911fa3c0805444483337064,2017-03,0.233044
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10,0.233044
4,0004aac84e0df4da2b147fca70cf8255,2017-11,0.233044


5.Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц. Описание подхода можно найти тут.

Retention: Retention дня X — это n процентов пользователей, которые вернутся к продукту в день X. В какой-то конкретный день (день 0) пришло 100 новых пользователей, а на 1-ый день вернулось 15, то Retention 15 / 100 = 15%.

In [22]:
df_order_customers = df_order.merge(df_customers, how='inner',on='customer_id')
# Наверно тут тоже стоит работать с пользователями совершившими покупку
df_order_customers = df_order_customers.query('order_status in ["delivered","shipped","processing"]')
# Сделаем дату
df_order_customers.order_approved_at = pd.to_datetime(df_order_customers.order_approved_at)
# Создадим дату первой покупки(когорту)
df_order_customers_first_buy = df_order_customers\
                                 .groupby('customer_unique_id', as_index=False)\
                                 .agg({'order_approved_at':'min'})\
                                 .rename(columns={'order_approved_at':'time_first_buy'})
df_order_customers = df_order_customers.merge(df_order_customers_first_buy, on='customer_unique_id')
# Приведем все к месяцу
df_order_customers.order_purchase_timestamp = pd.to_datetime(df_order_customers.order_purchase_timestamp).dt.strftime('%Y-%m')
df_order_customers.order_approved_at = pd.to_datetime(df_order_customers.order_approved_at).dt.strftime('%Y-%m')
df_order_customers.time_first_buy = df_order_customers['time_first_buy'].dt.strftime('%Y-%m')
# не уверен что это стоит делать, но удаляем пустые значения
df_order_customers = df_order_customers.dropna()
# создаем финальный df 
df_end = df_order_customers.query("'2017-01' <= time_first_buy and time_first_buy <= '2017-12'")\
                    .groupby(["time_first_buy", "order_approved_at"], as_index=False)\
                    .agg({"customer_unique_id": "nunique"})
# Все к дате
df_end["time_first_buy"] = pd.to_datetime(df_end["time_first_buy"]).dt.to_period("m")
df_end["order_approved_at"] = pd.to_datetime(df_end["order_approved_at"]).dt.to_period("m")
# Даем каждому месяцу номер начинаем с нулевого - месяц первой покупки. Используем attrgetter
df_end['period_month'] = (df_end.order_approved_at - df_end.time_first_buy).apply(attrgetter("n"))
# Переворачиваем, смотрим, считаем
df_end = df_end.pivot(index='time_first_buy', columns = 'period_month', values = 'customer_unique_id')
# Считаем Retention для 3-го месяца
round(df_end.div(df_end.iloc[:,0], axis = 0).mul(100), 2)[3].idxmax()

Period('2017-09', 'M')

6.Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию. В кластеризации можешь выбрать следующие метрики: R - время от последней покупки пользователя до текущей даты, F - суммарное количество покупок у пользователя за всё время, M - сумма покупок за всё время. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 2 до 5 заказов в неделю, monetary от 1780 до 3560 рублей в неделю. Описание подхода можно найти тут.

In [21]:
# Выбираем нужные фреймы данных
df_order_items = df_order.merge(df_items, how='inner', on='order_id')
df_order_items_users = df_order_items.merge(df_customers, how='inner', on='customer_id')
# Наверно тут тоже стоит работать с пользователями совершившими покупку
df_order_items_users = df_order_items_users.query('order_status in ["delivered","shipped","processing"]')
# Переводим в дату
df_order_items_users.order_approved_at = pd.to_datetime(df_order_items_users.order_approved_at)
# Создадим пару переменых с последней покупкой и сегодня
last_buy = df_order_items_users.order_approved_at.max()
today = pd.to_datetime(datetime.date.today())

In [22]:
# есть пустые значения - удалим
df_order_items_users.isnull().sum()
df_order_items_users = df_order_items_users.dropna()
# Создадим таблицу в соответствие с условием,где 
#                                              R - время от последней покупки пользователя до текущей даты, 
#                                              F - суммарное количество покупок у пользователя за всё время, 
#                                              M - сумма покупок за всё время
RFM = df_order_items_users\
    .groupby('customer_unique_id', as_index=False)\
    .agg({'order_approved_at': lambda x: (today - x.max()).days, 'order_id': 'nunique', 'price': lambda x: x.sum()})\
    .rename(columns={'order_approved_at': 'R', 'order_id': 'F', 'price': 'M'})

In [23]:
# Сегментация, сделаем 4 категории >25;25-50;50-75;75<
# Запишем значения в словарь
quantiles = RFM[['R', 'F', 'M']].quantile([.25, .5, .75]).to_dict()
quantiles

{'R': {0.25: 1800.0, 0.5: 1905.0, 0.75: 2032.0},
 'F': {0.25: 1.0, 0.5: 1.0, 0.75: 1.0},
 'M': {0.25: 47.65, 0.5: 89.7, 0.75: 154.6925}}

In [16]:
# Пропишем функцию для подсчета
# Для показателя Recency нумерация рангов в функции должна быть обратной, так как чем он меньше, тем лучше(меньше дней)
def R_score(x):
    if x <= quantiles['R'][.25]:
        return 4
    elif x <= quantiles['R'][.5]:
        return 3
    elif x <= quantiles['R'][.75]:
        return 2
    else:
        return 1
    
def M_score(x):
    if x <= quantiles['M'][.25]:
        return 1
    elif x <= quantiles['M'][.5]:
        return 2
    elif x <= quantiles['M'][.75]:
        return 3
    else:
        return 4  
# Для Frequency разбивка по квантилям не подходит, 
# так как большинство клиентов сделали всего одну покупку (границы квантилей у тебя везде равны
# 1 балл - 1 покупка
# 2 балла - 2 покупки
# 3 балла - 3-4 покупки
# 4 балла - 5 и более покупок    
def F_score(x):
    if x == 1:
        return 1
    elif x == 2:
        return 2
    elif 3 <= x <= 4:
        return 3
    else:
        return 4

In [24]:
RFM['Rseg'] = RFM['R'].apply(lambda x: R_score(x))
RFM['Fseg'] = RFM['F'].apply(lambda x: F_score(x))
RFM['Mseg'] = RFM['M'].apply(lambda x: M_score(x))
# Считаем RFM оценку
RFM['RFM_score'] = RFM['Rseg'].map(str) + RFM['Fseg'].map(str) + RFM['Mseg'].map(str)
RFM.head()

,customer_unique_id,R,F,M,Rseg,Fseg,Mseg,RFM_score
0,0000366f3b9a7992bf8c76cfdf3221e2,1798,1,129.90,4,1,3,413
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1801,1,18.90,3,1,1,311
2,0000f46a3911fa3c0805444483337064,2224,1,69.00,1,1,2,112
3,0000f6ccb0745a6a4b88665a16c9f078,2008,1,25.99,2,1,1,211
4,0004aac84e0df4da2b147fca70cf8255,1975,1,180.00,2,1,4,214


Выводы RFM_Score:
555 - Лояльные и активные, покупают много, часто и много платят;
333 - Покупали не очень давно, покупали средне, платили средне;
111 - Уходят, покупали давно, покупают редко и мало платят;
Нас больше всего интересуют [145],[245],[155],[255],[154],[254]
У указанных пользователей R=[1, 2], F=[4, 5], M=[4, 5] Они были лояльны, но потеряли активность (часто и много платили)